In [ ]:
import pandas as pd 
import numpy as np 
import os
import datetime as dt

In [ ]:
dfs = []
for f in os.listdir('raw/australia'):
    print(f)
    if f in ["hail.csv","waterspout.csv"]:
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,7),error_bad_lines=False)
    elif f == "rain.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,8),error_bad_lines=False)
    elif f == "ligthning.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,11),error_bad_lines=False)
    elif f == "tornado.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,9),error_bad_lines=False)
    elif f == "wind.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,10),error_bad_lines=False)
    df = pd.read_csv(f'raw/australia/{f}',error_bad_lines=False)
    dfs.append(df)

In [14]:
hail_df = dfs[0]
rain_df = dfs[1]
tornado_df = dfs[7]
lightning_df = dfs[8]
wind_df = dfs[9]
waterspout_df = dfs[12]
dffs = [dfs[6],dfs[2],dfs[3],dfs[4],dfs[5],dfs[10],dfs[11]]
for d in dffs[1:]:
    d['Year']=''
    yy = d.iloc[:,0]
    for i,n in yy.iteritems():
        n = str(n)
        count = 0
        year = []
        for a in n:
            if count < 4:
                year.append(a)
            count +=1
        year_final = ''.join(year)
        d['Year'][i] = str(year_final)

In [18]:
final_df = pd.DataFrame({'Year':[int(i) for i in range(1900,2021)]})
for d in dffs:
    d['Year'] = d['Year'].astype(int)
    final_df = final_df.merge(d,on="Year",how="left")
final_df = final_df[['Year', 'anomalie','max_temp','min_temp','rainfall_anom',
  'rainfall',' day_cloud', 'index']].\
    rename({'index':'drough_index', ' day_cloud': 'day_cloud'},axis=1).groupby('Year').mean()
final_df.head()

,anomalie,max_temp,min_temp,rainfall_anom,rainfall,day_cloud,drough_index
Year,,,,,,,
1900,NaN,NaN,NaN,-90.04,376.00,NaN,11.750000
1901,NaN,NaN,NaN,-61.12,404.92,NaN,9.333333
1902,NaN,NaN,NaN,-146.51,319.52,NaN,13.533333
1903,NaN,NaN,NaN,54.24,520.28,NaN,18.433333
1904,NaN,NaN,NaN,39.79,505.83,NaN,17.608333


In [19]:
final_df.tail()

,anomalie,max_temp,min_temp,rainfall_anom,rainfall,day_cloud,drough_index
Year,,,,,,,
2016,0.99,0.83,1.15,66.06,532.09,NaN,NaN
2017,1.06,1.41,0.72,28.41,494.44,NaN,NaN
2018,1.12,1.54,0.69,-60.19,405.85,NaN,NaN
2019,1.52,2.10,0.94,-190.32,275.71,NaN,NaN
2020,1.15,1.24,1.05,19.43,485.47,NaN,NaN


In [20]:
dfs_d = [hail_df,rain_df,tornado_df,lightning_df,wind_df,waterspout_df]
for df in dfs_d:
    df['date'] = pd.to_datetime(df['Date/Time'])
    df['Year'] = df['date'].dt.year

In [21]:
hail_df = hail_df[['Year', 'Hail size', 'Latitude', 'Longitude', 'Nearest town',
       'State']]
rain_df = rain_df[['Year','Intense precipitation amount', 'Intense precipitation period',
       'Total precipitation', 'Latitude', 'Longitude','State']]
lightning_df = lightning_df[['Year','Flash rate','Latitude', 'Longitude','State','Nearest town']]
tornado_df = tornado_df[['Year','Max speed (km/h)','Latitude', 'Longitude','State','Nearest town']]
wind_df = wind_df[['Year','Max Mean Wind speed','Latitude', 'Longitude','State','Nearest town']]
waterspout_df = waterspout_df[['Year','Number of Spouts','Latitude', 'Longitude']]
dfs_e = [hail_df,rain_df,tornado_df,lightning_df,wind_df,waterspout_df]

In [25]:
df2 = pd.DataFrame({'Year':[int(i) for i in range(1900,2021)]})
new_df = lightning_df.groupby('Year').count().iloc[:,0].reset_index()
new_df['Year'] = new_df['Year'].astype(int)
new_df2 = hail_df.groupby('Year').count().iloc[:,0].reset_index()
new_df2['Year'] = new_df2['Year'].astype(int)
new_df3 = rain_df.groupby('Year').mean().iloc[:,0].reset_index()
new_df3['Year'] = new_df3['Year'].astype(int)
new_df4 = tornado_df.groupby('Year').count().iloc[:,0].reset_index()
new_df4['Year'] = new_df4['Year'].astype(int)
new_df5 = waterspout_df.groupby('Year').count().iloc[:,0].reset_index()
new_df5['Year'] = new_df5['Year'].astype(int)
new_df6 = wind_df.groupby('Year').mean().iloc[:,0].reset_index()
new_df6['Year'] = new_df6['Year'].astype(int)
df2 = df2.merge(new_df,on="Year",how="left")
df2 = df2.merge(new_df2,on="Year",how="left")
df2 = df2.merge(new_df3,on="Year",how="left")
df2 = df2.merge(new_df4,on="Year",how="left")
df2 = df2.merge(new_df5,on="Year",how="left")
df2 = df2.merge(new_df6,on="Year",how="left")
df2 = df2.fillna(0)
summary_df = df2.rename({'Hail size':"hails_count", 'Intense precipitation amount': "rain_avg", 'Max speed (km/h)':"tornado_count",
       'Flash rate':'lightning_count', 'Max Mean Wind speed':"wind_speed_avg", 'Number of Spouts':'waterspouts_count'},axis=1)
summary_df.head()

,Year,lightning_count,hails_count,rain_avg,tornado_count,waterspouts_count,wind_speed_avg
0,1900,0.0,5.0,31.400000,3.0,0.0,0.0
1,1901,1.0,7.0,81.600000,7.0,0.0,0.0
2,1902,0.0,11.0,81.666667,7.0,0.0,0.0
3,1903,0.0,10.0,60.842105,9.0,0.0,0.0
4,1904,1.0,5.0,60.375000,6.0,0.0,0.0


In [26]:
ff_df = final_df.merge(summary_df,on="Year",how="left")
ff_df['Year'] = ff_df['Year'].astype(int)
ff_df.fillna(0,inplace=True)

In [29]:
final_df = ff_df[['Year','anomalie','max_temp', 'min_temp', 
'rainfall_anom','rainfall','day_cloud', 'drough_index', 'lightning_count', 'hails_count',
       'rain_avg', 'tornado_count', 'waterspouts_count', 'wind_speed_avg']].\
           rename({'anomalie':'avg_temp'},axis=1)

In [31]:
final_df = final_df[final_df['Year']>= 1910]

In [48]:
final_df.to_csv('cleaned/australia_extreme_summary.csv',index=False)
